<a href="https://colab.research.google.com/github/oanhnguyen-mba/htmljs-interactive-financial-kpi-dashboard/blob/main/demo_financial_KPI_dashboard_interactive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
%%writefile financial_kpi_dashboard.html

<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <title>Financial KPI Dashboard Interactive</title>

  <!-- Load math.js for calculations if needed -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/mathjs/11.8.0/math.min.js"></script>
  <!-- Load Chart.js for charts -->
  <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>

  <style>
    body {
      font-family: Arial, sans-serif;
      margin: 30px;
      max-width: 900px;
    }
    h1 {
      color: #4CAF50;
    }
    table {
      border-collapse: collapse;
      width: 100%;
      margin-bottom: 20px;
    }
    th, td {
      border: 1px solid #ddd;
      padding: 8px;
      text-align: center;
    }
    th {
      background-color: #f2f2f2;
    }
    input[type="number"] {
      width: 100px;
      font-size: 1em;
    }
    .btn {
      background-color: #4CAF50;
      color: white;
      padding: 10px 20px;
      border: none;
      cursor: pointer;
      margin-right: 10px;
      font-size: 1em;
      border-radius: 4px;
    }
    .btn:hover {
      background-color: #45a049;
    }
    .result {
      font-size: 1.2em;
      margin-top: 20px;
      background: #e9f5e9;
      border: 1px solid #b2d8b2;
      padding: 15px;
      border-radius: 8px;
    }
    #charts {
      margin-top: 40px;
    }
  </style>
</head>
<body>
  <h1>📊 Financial KPI Dashboard Interactive</h1>
  <p>Enter your financial data and discount rate, then click "Calculate KPIs" to see results and charts instantly.</p>

  <!-- Financial Data Table -->
  <table id="data-table">
    <thead>
      <tr>
        <th>Year</th>
        <th>Revenue ($)</th>
        <th>Expenses ($)</th>
        <th>Investment ($)</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td><input type="number" value="2021" /></td>
        <td><input type="number" value="100000" /></td>
        <td><input type="number" value="60000" /></td>
        <td><input type="number" value="-150000" /></td>
      </tr>
      <tr>
        <td><input type="number" value="2022" /></td>
        <td><input type="number" value="120000" /></td>
        <td><input type="number" value="65000" /></td>
        <td><input type="number" value="0" /></td>
      </tr>
      <tr>
        <td><input type="number" value="2023" /></td>
        <td><input type="number" value="130000" /></td>
        <td><input type="number" value="70000" /></td>
        <td><input type="number" value="0" /></td>
      </tr>
      <tr>
        <td><input type="number" value="2024" /></td>
        <td><input type="number" value="140000" /></td>
        <td><input type="number" value="72000" /></td>
        <td><input type="number" value="-10000" /></td>
      </tr>
      <tr>
        <td><input type="number" value="2025" /></td>
        <td><input type="number" value="150000" /></td>
        <td><input type="number" value="75000" /></td>
        <td><input type="number" value="-5000" /></td>
      </tr>
    </tbody>
  </table>

  <!-- Buttons and Discount Rate Input -->
  <label for="discountRate">Discount Rate (%): </label>
  <input type="number" id="discountRate" value="10" step="0.1" min="0" max="100" />

  <button class="btn" onclick="addRow()">➕ Add Year</button>
  <button class="btn" onclick="calculateKPIs()">✅ Calculate KPIs</button>

  <!-- KPI Summary and Insight -->
  <div class="result" id="kpi-summary"></div>

  <!-- Charts Section -->
  <div id="charts">
    <canvas id="barChart" height="200"></canvas>
    <canvas id="lineChart" height="200" style="margin-top:30px;"></canvas>
  </div>

  <script>
    let barChart, lineChart;

    // Function to add a new year row in the table
    function addRow() {
      const tableBody = document.querySelector("#data-table tbody");
      const newRow = document.createElement("tr");
      for(let i=0; i<4; i++) {
        const cell = document.createElement("td");
        cell.innerHTML = '<input type="number" value="0" />';
        newRow.appendChild(cell);
      }
      tableBody.appendChild(newRow);
    }

    // Main function to calculate KPIs and update dashboard
    function calculateKPIs() {
      // Get discount rate from input and convert percentage to decimal
      const discountRateInput = document.getElementById("discountRate").value;
      const discountRate = parseFloat(discountRateInput) / 100;

      // Extract data from table
      const rows = document.querySelectorAll("#data-table tbody tr");
      let years = [], revenue = [], expenses = [], investments = [], cashflows = [];
      let totalInvestment = 0;

      rows.forEach(row => {
        const inputs = row.querySelectorAll("input");
        const year = inputs[0].value;
        const rev = parseFloat(inputs[1].value) || 0;
        const exp = parseFloat(inputs[2].value) || 0;
        const inv = parseFloat(inputs[3].value) || 0;
        const cashflow = rev - exp + inv;

        years.push(year);
        revenue.push(rev);
        expenses.push(exp);
        investments.push(inv);
        cashflows.push(cashflow);

        if(inv < 0) totalInvestment += Math.abs(inv);
      });

      // Calculate NPV using discount rate
      let npv = 0;
      cashflows.forEach((cf, t) => {
        npv += cf / Math.pow(1 + discountRate, t);
      });

      // Simple IRR calculation by iterative method
      function irr(values, guess=0.1) {
        const tol = 1e-6;
        const maxIter = 1000;
        let rate = guess;
        for(let i=0; i<maxIter; i++) {
          let npvCalc = values.reduce((acc, val, t) => acc + val / Math.pow(1 + rate, t), 0);
          if(Math.abs(npvCalc) < tol) return rate;
          rate += 0.0001;
        }
        return NaN;
      }
      const irrValue = irr(cashflows) * 100;

      // ROI calculation: total profit over total investment in percentage
      const totalProfit = cashflows.reduce((a,b) => a + b, 0) - totalInvestment;
      const roi = (totalProfit / totalInvestment) * 100;

      // Calculate payback period (years until cumulative cashflow >= 0)
      let cumulativeCashflow = 0;
      let paybackYear = null;
      for(let i=0; i<cashflows.length; i++) {
        cumulativeCashflow += cashflows[i];
        if(cumulativeCashflow >= 0) {
          paybackYear = years[i];
          break;
        }
      }

      // Create insight text based on KPI results
      let insightText = "";
      if(npv > 0) {
        insightText += "✅ Project is profitable (NPV > 0).<br>";
      } else {
        insightText += "❌ Project results in a loss (NPV < 0).<br>";
      }
      if(paybackYear !== null) {
        insightText += `💰 Payback period: approximately by year ${paybackYear}.<br>`;
      } else {
        insightText += "⚠ Payback period exceeds project duration.<br>";
      }

      // Display KPI summary and insights
      document.getElementById("kpi-summary").innerHTML = `
        <b>📈 Financial KPIs Summary:</b><br>
        NPV: $${npv.toFixed(2)}<br>
        IRR: ${irrValue.toFixed(2)}%<br>
        ROI: ${roi.toFixed(2)}%<br><br>
        <b>💡 Insights:</b><br>
        ${insightText}
      `;

      // Draw Bar Chart (Revenue, Expenses, Investment)
      if(barChart) barChart.destroy();
      barChart = new Chart(document.getElementById("barChart"), {
        type: 'bar',
        data: {
          labels: years,
          datasets: [
            { label: "Revenue", data: revenue, backgroundColor: 'rgba(102,194,165,0.7)' },       // Pastel green
            { label: "Expenses", data: expenses, backgroundColor: 'rgba(252,141,98,0.7)' },      // Pastel red
            { label: "Investment", data: investments, backgroundColor: 'rgba(141,160,203,0.7)' } // Pastel blue
          ]
        },
        options: {
          responsive: true,
          plugins: {
            title: {
              display: true,
              text: 'Revenue, Expenses & Investment (Pastel Colors)'
            }
          },
          scales: {
            y: { beginAtZero: true }
          }
        }
      });

      // Draw Line Chart for Net Cashflow
      if(lineChart) lineChart.destroy();
      lineChart = new Chart(document.getElementById("lineChart"), {
        type: 'line',
        data: {
          labels: years,
          datasets: [{
            label: "Net Cashflow",
            data: cashflows,
            borderColor: 'rgba(100, 149, 237, 1)',    // Cornflower Blue
            backgroundColor: 'rgba(100, 149, 237, 0.2)',
            fill: true,
            tension: 0.3
          }]
        },
        options: {
          responsive: true,
          plugins: {
            title: {
              display: true,
              text: 'Net Cashflow Over Years'
            }
          },
          scales: {
            y: { beginAtZero: true }
          }
        }
      });
    }
  </script>
</body>
</html>



Overwriting financial_kpi_dashboard.html


In [6]:
# Download file
from google.colab import files
files.download('financial_kpi_dashboard.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>